In [ ]:
import astropy.io.fits as fits
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from astropy.cosmology import Planck15 as cosmo

from galaxy_cluster_matching import match_galaxies_and_clusters
from completeness_new import calculate_completeness_of_objects
from mass_function import get_weighted_mass_histogram, get_region_volume
from constants import MASS_BINS, Z_MAX, LOG_MASS_LUMINOSITY_RATIO_BINS
from richard_curve import get_mass_luminosity_cutoff, get_distance_from_mass, filter_for_richards_curve, get_mass_luminosity_histogram
from constants import MASS_BINS, Z_MAX, LOG_MASS_LUMINOSITY_RATIO_BINS

In [29]:
'''
todo: mix the correct files,
    get the correct name of the columns
                            and masks
    get the area for SDSS
    and also area of its patch on the sky for the VOLUME calculation (ask johan)

'''

'\ntodo: mix the correct files,\n    get the correct name of the columns\n                            and masks\n    get the area for SDSS\n    and also area of its patch on the sky for the VOLUME calculation (ask johan)\n\n'

In [30]:
# SDSS_All_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/SDSS/NYU/object_sdss_spectro.fits')[1].data
# SDSS_All_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/SDSS/SDSS_DR7/SDSS_DR7.fits')[1].data  #galaxy
SDSS_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/SDSS/SDSS_DR7/MPA/merged/gal_merged_info_mass.fit')[1].data  #galaxy

eRASS1 = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/eRASS/merged_primary&optical_clusters/merged_optical_primary_clusters.fits')[1].data #cluster

In [68]:
# Big galaxy catalog
sdss_full_df = pd.DataFrame({
    'RA': SDSS_raw['RA'].byteswap().newbyteorder(),
    'DEC': SDSS_raw['DEC'].byteswap().newbyteorder(),
    'z': SDSS_raw['Z'].byteswap().newbyteorder(),
    'mstar': SDSS_raw['MEDIAN'].byteswap().newbyteorder(), # -1 is the NAN, and the values are like 10.294701,  which means M* =10^(10.294701)M☉
    'plugmag_r': SDSS_raw['PLUG_MAG'][:, 2].byteswap().newbyteorder(),  # Extract r-band
    'specmag_r': SDSS_raw['SPECTRO_MAG'][:, 2].byteswap().newbyteorder(),  # Extract r-band
})

# Derive flux in Jansky (Jy)
m0 = -48.6
sdss_full_df['flux'] = 10 ** (-0.4 * (sdss_full_df['specmag_r'] - m0)) * 1e23
sdss_full_df['flux_plugmag'] = 10 ** (-(sdss_full_df['plugmag_r'] + 48.6) / 2.5) * 1e23
# Calculate comoving distance from redshift
sdss_full_df['comoving_distance'] = cosmo.comoving_distance(sdss_full_df['z']).value




# Galaxy Masks
sdss_full_df = sdss_full_df[
    (sdss_full_df['z'] != 0) &                   # Redshift not zero
    (sdss_full_df['z'] <= Z_MAX) &                # Redshift less than Z_MAX (define Z_MAX)
    (sdss_full_df['flux'] >= 5.011928e-05) &       # Flux greater than or equal to 5.011928e-05
    (sdss_full_df['flux_plugmag'] >= 5.011928e-05)       # Flux greater than or equal to 5.011928e-05
]


# SMALL galaxy catalog
sdss_df = sdss_full_df[sdss_full_df['mstar'] != -1].copy()


In [70]:
print(sdss_df)


                RA        DEC         z      mstar  plugmag_r  specmag_r  \
0       146.714203  -1.041304  0.021275  10.294701  16.304701  15.627859   
1       146.919449  -0.990492  0.213925  11.162018  19.082600  18.079983   
2       146.859833  -0.808902  0.126554  11.373624  18.942801  17.694822   
4       146.963898  -0.545003  0.055988   9.958836  19.477200  18.517302   
5       146.607208  -1.240753  0.090633  10.915439  18.250299  17.301632   
...            ...        ...       ...        ...        ...        ...   
926952  258.599121  32.235626  0.078880  10.462537  18.752800  17.908915   
927342  259.786102  30.735430  0.279855  11.617262  19.014900  17.876860   
927350  260.124939  29.922937  0.122983  10.816550  18.541401  17.728378   
927353  259.341309  30.198423  0.146299  10.950871  18.193899  17.409668   
927355  259.667084  30.420561  0.048978   9.382871  19.091299  18.062140   

            flux  flux_plugmag  comoving_distance  
0       0.002036      0.001092     

In [71]:
# load the clusters of galaxies data from the fits object into dataframes
cluster_df_raw = pd.DataFrame({
    'c_ID': eRASS1['DETUID'].byteswap().newbyteorder(),
    'c_NAME': eRASS1['NAME'].byteswap().newbyteorder(),
    'RA': eRASS1['RA'].byteswap().newbyteorder(),
    'DEC': eRASS1['DEC'].byteswap().newbyteorder(),
    'z': eRASS1['BEST_Z'].byteswap().newbyteorder(),
    'z_type': eRASS1['BEST_Z_TYPE_1'].byteswap().newbyteorder(),
    'cluster_radius_kpc': eRASS1['R500'].byteswap().newbyteorder(),
    'cluster_Velocity_Dispersion': eRASS1['VDISP_BOOT'].byteswap().newbyteorder(),
})

In [72]:
# Masks
# CLUSTERS
# Remove rows where 'cluster_Velocity_Dispersion' is NaN
# cluster_df_noVelDisp = cluster_df_raw.dropna(subset=['cluster_Velocity_Dispersion'])

cluster_df = cluster_df_raw[
    (cluster_df_raw['z'] <= 0.4) &
    (cluster_df_raw['cluster_radius_kpc'] != -1)
]

cluster_df['cluster_radius_Mpc'] = cluster_df['cluster_radius_kpc'] / 1000


/tmp/ipykernel_140860/3430907919.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df['cluster_radius_Mpc'] = cluster_df['cluster_radius_kpc'] / 1000


In [73]:
print(cluster_df)
print(cluster_df_raw)
print(len(cluster_df)/len(cluster_df_raw))

                                   c_ID                   c_NAME          RA  \
0      em01_002129_020_ML00014_002_c010  1eRASS J000005.2-383729    0.021730   
1      em01_011174_020_ML00047_002_c010  1eRASS J000021.4-824954    0.089191   
2      em01_002141_020_ML00031_002_c010  1eRASS J000031.0-500903    0.129422   
3      em01_003153_020_ML00092_010_c010  1eRASS J000031.8-630131    0.132712   
4      em01_002135_020_ML00092_002_c010  1eRASS J000045.3-462341    0.189080   
...                                 ...                      ...         ...   
12241  em01_357150_020_ML00035_011_c010  1eRASS J235853.5-612935  359.723163   
12242  em01_357150_020_ML00008_011_c010  1eRASS J235858.0-603644  359.741833   
12243  em01_358144_020_ML00091_010_c010  1eRASS J235911.7-551748  359.799123   
12244  eb01_358123_020_ML00022_002_c010  1eRASS J235927.5-341821  359.864701   
12246  em01_358129_020_ML00004_002_c010  1eRASS J235956.1-392826  359.983837   

             DEC        z       z_type 

In [74]:
sdss_df = calculate_completeness_of_objects(sdss_full_df, sdss_df) # for sdss the complteness is now defined the galaxies that their mass is estimated

/home/farnoosh/git_repos/Galaxy-Stellar-Mass-Function-in-X-ray-Selected-Clusters/completeness_new.py:23: RuntimeWarning: invalid value encountered in divide
  return np.cumsum(number_of_obj_small_survey)/np.cumsum(number_of_obj_big_survey)


In [75]:
print(sdss_df)

                RA        DEC         z      mstar  plugmag_r  specmag_r  \
0       146.714203  -1.041304  0.021275  10.294701  16.304701  15.627859   
1       146.919449  -0.990492  0.213925  11.162018  19.082600  18.079983   
2       146.859833  -0.808902  0.126554  11.373624  18.942801  17.694822   
4       146.963898  -0.545003  0.055988   9.958836  19.477200  18.517302   
5       146.607208  -1.240753  0.090633  10.915439  18.250299  17.301632   
...            ...        ...       ...        ...        ...        ...   
926952  258.599121  32.235626  0.078880  10.462537  18.752800  17.908915   
927342  259.786102  30.735430  0.279855  11.617262  19.014900  17.876860   
927350  260.124939  29.922937  0.122983  10.816550  18.541401  17.728378   
927353  259.341309  30.198423  0.146299  10.950871  18.193899  17.409668   
927355  259.667084  30.420561  0.048978   9.382871  19.091299  18.062140   

            flux  flux_plugmag  comoving_distance  completeness  
0       0.002036     

In [76]:
matched_sdss_dataframe = match_galaxies_and_clusters(galaxy_dataframe=sdss_df.head(1000), cluster_dataframe=cluster_df.head(10))
matched_sdss_dataframe.to_csv('sdss_matched_in_erass1.csv')

Progress: 16.6%0000000000002%

/home/farnoosh/git_repos/Galaxy-Stellar-Mass-Function-in-X-ray-Selected-Clusters/galaxy_cluster_matching.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #     galaxy_dataframe['environment'] = "Field"
/home/farnoosh/git_repos/Galaxy-Stellar-Mass-Function-in-X-ray-Selected-Clusters/galaxy_cluster_matching.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #     galaxy_dataframe['cluster_mass'] = None
/home/farnoosh/git_repos/Galaxy-Stellar-Mass-Function-in-X-ray-Selected-Clusters/galaxy_cluster_matchi

Progress: 117.6%999999999999%

In [77]:
# load matched galaxies
sdss_df = pd.read_csv('sdss_matched_in_erass1.csv')

In [78]:
# apply selection function
mass_to_light_histogram_all_galaxies = get_mass_luminosity_histogram(galaxy_df=sdss_df)
mass_to_light_histogram_low_mass_galaxies = get_mass_luminosity_histogram(galaxy_df=sdss_df[sdss_df["mstar"] < 10**10])
mass_luminosity_cutoff = get_mass_luminosity_cutoff(sdss_df, cut_off_percentage=80)
mass_bin_selection_function = np.logspace(5,12.25, 1000) # todo: might need a change
selection_function = get_distance_from_mass(mass_bin_selection_function, log_cutoff_mass_to_light_ratio=mass_luminosity_cutoff)
sdss_df['within_selection_function'] = sdss_df.apply(lambda row: filter_for_richards_curve(selection_function, mass_bin_selection_function, row['mstar'], row['comoving_distance']), axis=1)

In [79]:
# find the general mass function
mass_histogram, mass_histogram_error = get_weighted_mass_histogram(sdss_df[sdss_df['within_selection_function']], region_name='sdss_x_erass')
stellar_mass_function = mass_histogram / get_region_volume('sdss_x_erass', MASS_BINS[:-1] , mass_luminosity_cutoff)
stellar_mass_function_error = mass_histogram_error / get_region_volume('sdss_x_erass', MASS_BINS[:-1] , mass_luminosity_cutoff)

KeyError: 'sdss_x_erass'